<a href="https://colab.research.google.com/github/Carlcarrie/Carlcarrie/blob/main/stStockProphet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
! pip install streamlit --quiet

In [10]:
! pip install yfinance --quiet

In [30]:
! pip install NeuralProphet --quiet

     |████████████████████████████████| 67 kB 2.7 MB/s 
     |████████████████████████████████| 155 kB 5.6 MB/s 


In [1]:
%%writefile app.py
import streamlit as st
from datetime import date

import yfinance as yf
from fbprophet import Prophet
from fbprophet.plot import plot_plotly
from plotly import graph_objs as go

START = "2018-01-01"
TODAY = date.today().strftime("%Y-%m-%d")

st.title('Stock Forecast - Crude, to illustrate Tunneling Streamlit over Colab')
st.header('using Univariate Meta Prophet Model on Daily Stock Close Price Data')
#st.subheader('and Google Colab and Localtunnel for rendering Streamlit Code')

stocks = ('AAL',
          'AAPL',
          'BAC',
          'CCL',
          'DIDI',
          'GOOG',
          'GME',
          'F',
          'MSFT',
          'NIO',
          'NVDA',
          'PFE',
          'SOFI',
          'T'
)
selected_stock = st.selectbox('Select stock', stocks)

period = st.slider('How many days to Predict', 1, 100)

@st.cache
def load_data(ticker):
    data = yf.download(ticker, START, TODAY)
    data.reset_index(inplace=True)
    return data

    
data_load_state = st.text('Loading data...')
data = load_data(selected_stock)
data_load_state.text('Loading data... done!')

st.subheader('Data Table of most recent 5-days of OHLC Stock Prices')
st.write(data.tail())

# Plot raw data
def plot_raw_data():
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=data['Date'], y=data['Adj Close'], name="stock_close"))
    fig.layout.update(title_text=f'Time Series of {selected_stock} Adjusted Close Prices', xaxis_rangeslider_visible=True)
    st.plotly_chart(fig)
    
plot_raw_data()

# Predict forecast with Prophet.
df_train = data[['Date','Close']]
df_train = df_train.rename(columns={"Date": "ds", "Close": "y"})

m = Prophet()
m.fit(df_train)
future = m.make_future_dataframe(periods=period)
forecast = m.predict(future)

# Show and plot forecast
st.subheader(f'Forecast data for 5-day recent subset of {period} rolling days of predictions')
st.write(forecast.tail())
    
st.write(f'Forecast plot for {period} days')
fig1 = plot_plotly(m, forecast)
st.plotly_chart(fig1)

st.write("Forecast components")
fig2 = m.plot_components(forecast)
st.write(fig2)

st.write('###### End #######')


Overwriting app.py


In [ ]:
! streamlit run app.py & npx localtunnel --port 8501


  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://35.236.149.223:8501

npx: installed 22 in 6.126s
2021-12-29 21:50:55.484 NumExpr defaulting to 2 threads.
your url is: https://serious-lionfish-6.loca.lt
[*********************100%***********************]  1 of 1 completed
2021-12-29 21:51:00.727 Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
Initial log joint probability = -18.6761
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       2472.08      0.027197       1711.91           1           1      117   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       2533.91   0.000998069       158.518           1           1      229   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299       2549.33     0.0193854       272.095       